In [2]:
import numpy as np
import pandas as pd
import sqlite3
import optuna
import time
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import japanize_matplotlib
from IPython.display import display, HTML
from epics import caput


/home/skato/anaconda3/envs/optuna/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# .dbファイルのパスを指定
db_path = 'SKEKB20240529.db'

# SQLクエリを使ってデータを読み込む
query = "SELECT * FROM trial_user_attributes"

# SQLiteデータベースに接続
conn = sqlite3.connect(db_path)

# 'trials' テーブルのデータを読み込む
df_constraints = pd.read_sql_query(query, conn)

# CSVとして保存
df_constraints.to_csv("Trial_constraints.csv", index=False)

# 接続を閉じる
conn.close()

In [ ]:
import pandas as pd

# CSVファイルのパス
input_csv_path = '/mnt/data/Trial_constraints.csv'
output_csv_path = '/mnt/data/Extracted_value_json.csv'

# CSVファイルを読み込む
df = pd.read_csv(input_csv_path)

# 'value_json' 列を抽出
value_json_df = df[['value_json']]

# 新しいCSVファイルとして保存
value_json_df.to_csv(output_csv_path, index=False)

print(f"'value_json' column has been extracted and saved to {output_csv_path}")

In [12]:
import pandas as pd
import json

# CSVファイルのパス
input_csv_path = 'Trial_constraints.csv'
output_csv_path = 'constraints.csv'

# CSVファイルを読み込む
df = pd.read_csv(input_csv_path)

# 'value_json' 列から数値を抽出する関数
def extract_numbers(json_str):
    try:
        data = json.loads(json_str)
        if isinstance(data, dict):
            # 辞書の場合
            return [value for value in data.values() if isinstance(value, (int, float))]
        elif isinstance(data, list):
            # リストの場合
            return [item for item in data if isinstance(item, (int, float))]
        else:
            return []
    except json.JSONDecodeError:
        return []

# 'value_json' 列から数値を抽出
df['extracted_numbers'] = df['value_json'].apply(extract_numbers)

# 数値のリストを展開して新しいDataFrameを作成
extracted_numbers_df = df['extracted_numbers'].apply(pd.Series)

# 新しいCSVファイルとして保存
extracted_numbers_df.to_csv(output_csv_path, index=False, header=False)

print(f"Extracted numbers have been saved to {output_csv_path}")

Extracted numbers have been saved to constraints.csv


In [9]:
#クエリ確認用のコード

query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql_query(query_tables, conn)
print("Tables in the database:")
print(tables_df)

ProgrammingError: Cannot operate on a closed database.